# Data Loading (MobileNetV3 Large)

In [ ]:
# ==============================================================================
# PROSES 1: PERSIAPAN DATA (KHUSUS UNTUK MOBILENETV3)
# ==============================================================================

# Langkah 1.1: Install Library
!pip install -q albumentations

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from google.colab import drive
from collections import Counter
import albumentations as A
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import random

# Langkah 1.2: Mount Google Drive
drive.mount('/content/drive')

# Langkah 1.3: Definisi Path dan Parameter
DATASET_DIR = '/content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15'
# DATASET_DIR = '/content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15_6_transformasi'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32


SEED_VALUE = 42
def set_seed(seed=42):
  """
  Mengatur seed dan menonaktifkan parallelism untuk hasil yang fully deterministic.
  """
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

  # Menjalankan operasi TF pada satu thread untuk menghilangkan randomness dari parallelism
  tf.config.threading.set_inter_op_parallelism_threads(1)
  tf.config.threading.set_intra_op_parallelism_threads(1)

  print(f"Random seed set to {seed} and parallelism disabled for reproducibility.")

set_seed(SEED_VALUE)

# Langkah 1.4: Ambil Nama Kelas dari Struktur Direktori Train
train_base_dir = os.path.join(DATASET_DIR, 'train')
class_dirs = sorted([d for d in os.listdir(train_base_dir) if os.path.isdir(os.path.join(train_base_dir, d))])
class_map = {name: i for i, name in enumerate(class_dirs)}

class_names = class_dirs
print(f"Nama kelas: {class_names}")
NUM_CLASSES = len(class_names)
print(f"Jumlah kelas: {NUM_CLASSES}")

# Global lists untuk verifikasi distribusi data
train_labels, val_labels, test_labels = [], [], []

# Langkah 1.5: Transformasi Gambar
basic_transform = A.Compose([
    A.Resize(height=IMAGE_SIZE[0], width=IMAGE_SIZE[1])
])

# Langkah 1.6: Custom Data Generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, class_map, batch_size, transform, num_classes, shuffle=True, store_global_labels=None):
        self.directory = directory
        self.class_map = class_map
        self.batch_size = batch_size
        self.transform = transform
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.image_paths = []
        self.labels = []
        self._load_image_paths()
        self.on_epoch_end()

        if store_global_labels is not None:
            store_global_labels.extend(self.labels)

    def _load_image_paths(self):
        """Memuat path gambar dari direktori yang diberikan."""
        print(f"Loading images from: {self.directory}")
        for class_name in sorted(self.class_map.keys()):
            class_path = os.path.join(self.directory, class_name)
            if os.path.isdir(class_path):
                class_idx = self.class_map[class_name]
                for fname in sorted(os.listdir(class_path)):
                    if fname.lower().endswith('.jpg'):
                        fpath = os.path.join(class_path, fname)
                        self.image_paths.append(fpath)
                        self.labels.append(class_idx)

    def __len__(self):
        """Menghitung jumlah batch per epoch."""
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        """Menghasilkan satu batch data."""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_paths = [self.image_paths[i] for i in indexes]
        batch_labels = [self.labels[i] for i in indexes]
        batch_images = []

        for path in batch_paths:
            img = Image.open(path).convert('RGB')
            img_cv = np.array(img)
            transformed = self.transform(image=img_cv) # Resize gambar
            batch_images.append(transformed['image'])

        # Preprocessing khusus MobileNetV3 Large
        batch_images = preprocess_input(np.array(batch_images, dtype=np.float32))
        batch_labels = tf.keras.utils.to_categorical(batch_labels, num_classes=self.num_classes)
        return batch_images, batch_labels

    def on_epoch_end(self):
        """Mengacak indeks di akhir setiap epoch."""
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Langkah 1.7: Inisialisasi Generator
train_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'train'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=True,
    store_global_labels=train_labels
)

val_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'val'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=False,
    store_global_labels=val_labels
)

test_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'test'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=False,
    store_global_labels=test_labels
)

print(f"\nTotal gambar train: {len(train_generator.image_paths)}")
print(f"Total gambar validation: {len(val_generator.image_paths)}")
print(f"Total gambar test: {len(test_generator.image_paths)}")

print("\nPROSES 1 SELESAI: Data MobileNetV3 siap dari direktori yang sudah di-augmentasi.")


# ==============================================================================
# Verifikasi Distribusi Data di Setiap Split
# ==============================================================================
print("\nVerifikasi Distribusi Data:")

# Menghitung jumlah kemunculan setiap label di setiap set
train_counts = Counter(train_labels)
val_counts = Counter(val_labels)
test_counts = Counter(test_labels)

# Membuat tabel untuk visualisasi yang mudah dibaca
distribusi_data = {
    "Kelas": class_names,
    "Jumlah Train": [train_counts.get(i, 0) for i in range(len(class_names))],
    "Jumlah Validasi": [val_counts.get(i, 0) for i in range(len(class_names))],
    "Jumlah Test": [test_counts.get(i, 0) for i in range(len(class_names))],
}
df_distribusi = pd.DataFrame(distribusi_data)
df_distribusi["Total per Kelas"] = df_distribusi["Jumlah Train"] + df_distribusi["Jumlah Validasi"] + df_distribusi["Jumlah Test"]

# Menambahkan baris total di bagian bawah
total_row = {
    "Kelas": "TOTAL",
    "Jumlah Train": sum(df_distribusi["Jumlah Train"]),
    "Jumlah Validasi": sum(df_distribusi["Jumlah Validasi"]),
    "Jumlah Test": sum(df_distribusi["Jumlah Test"]),
    "Total per Kelas": sum(df_distribusi["Total per Kelas"])
}
df_distribusi = pd.concat([df_distribusi, pd.DataFrame([total_row])], ignore_index=True)

print("Distribusi jumlah data per kelas di setiap split:")
print(df_distribusi.to_string())

Mounted at /content/drive
Random seed set to 42 and parallelism disabled for reproducibility.
Nama kelas: ['A 801 T', 'BL', 'BM', 'CLS 02', 'JF 03 TA', 'JF 08 ST', 'K 406 CTG', 'K 407 MH', 'K 409 GWC', 'V 688 CA', 'V 697 GKU', 'V TUL', 'Y 316 FA', 'Y 321 C', 'Y 327 GKU']
Jumlah kelas: 15
Loading images from: /content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15/train
Loading images from: /content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15/val
Loading images from: /content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15/test

Total gambar train: 3000
Total gambar validation: 301
Total gambar test: 320

PROSES 1 SELESAI: Data MobileNetV3 siap dari direktori yang sudah di-augmentasi.

Verifikasi Distribusi Data:
Distribusi jumlah data per kelas di setiap split:
        Kelas  Jumlah Train  Jumlah Validasi  Jumlah Test  Total per Kelas
0     A 801 T           200               16           17              233
1          BL           200               15           17    

# MobileNetV3 Large Tanpa Decay

In [ ]:
# ==============================================================================
# PROSES 2 & 3: PEMBANGUNAN, PELATIHAN, EVALUASI, DAN PENYIMPANAN MODEL
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV3Large
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Daftar nilai dropout yang akan diuji
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Direktori utama untuk menyimpan hasil
base_output_dir = '/content/drive/MyDrive/hasil_eksperimen_dropout_mobilenetv3_large'
os.makedirs(base_output_dir, exist_ok=True)

for dropout_rate in dropout_values:
    print(f"\n{'='*80}")
    print(f"MEMULAI EKSPERIMEN DENGAN DROPOUT = {dropout_rate} (MobileNetV3-Large)")
    print(f"{'='*80}\n")

    output_dir = os.path.join(base_output_dir, f'dropout_{dropout_rate}')
    os.makedirs(output_dir, exist_ok=True)

    base_model = MobileNetV3Large(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, seed=SEED_VALUE)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model_mobilenet_final = Model(inputs=inputs, outputs=outputs)
    model_mobilenet_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    print("Arsitektur Model:")
    model_mobilenet_final.summary()

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\nMemulai pelatihan model dengan dropout = {dropout_rate}...")
    history = model_mobilenet_final.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping], verbose=1)
    print(f"\nPelatihan model dengan dropout = {dropout_rate} telah selesai.")

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1); plt.plot(history.history['accuracy'], label='Akurasi Pelatihan'); plt.plot(history.history['val_accuracy'], label='Akurasi Validasi'); plt.title(f'Grafik Akurasi (Dropout = {dropout_rate})'); plt.xlabel('Epoch'); plt.ylabel('Akurasi'); plt.legend()
    plt.subplot(1, 2, 2); plt.plot(history.history['loss'], label='Loss Pelatihan'); plt.plot(history.history['val_loss'], label='Loss Validasi'); plt.title(f'Grafik Loss (Dropout = {dropout_rate})'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plot_path = os.path.join(output_dir, 'grafik_pelatihan.png'); plt.savefig(plot_path); plt.close(); print(f"Grafik pelatihan telah disimpan di: {plot_path}")

    print("\nMelakukan evaluasi final pada data tes...")
    test_loss, test_accuracy = model_mobilenet_final.evaluate(test_generator); print(f"Akurasi pada data tes (Dropout = {dropout_rate}): {test_accuracy:.4f}")

    y_pred_probs = model_mobilenet_final.predict(test_generator); y_pred = np.argmax(y_pred_probs, axis=1); y_true = np.array(test_labels)

    print("\nLaporan Klasifikasi pada Data Tes:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True); report_df = pd.DataFrame(report).transpose(); report_path = os.path.join(output_dir, 'classification_report.csv'); report_df.to_csv(report_path); print(classification_report(y_true, y_pred, target_names=class_names)); print(f"Laporan klasifikasi telah disimpan di: {report_path}")
    except Exception as e:
        print(f"Tidak dapat membuat laporan klasifikasi: {e}")

    print("\nConfusion Matrix pada Data Tes:")
    cm = confusion_matrix(y_true, y_pred); plt.figure(figsize=(10, 8)); sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names); plt.title(f'Confusion Matrix (Dropout = {dropout_rate})'); plt.ylabel('Kelas Sebenarnya'); plt.xlabel('Kelas Prediksi'); cm_path = os.path.join(output_dir, 'confusion_matrix.png'); plt.savefig(cm_path); plt.close(); print(f"Confusion matrix telah disimpan di: {cm_path}")

    print("\nMemulai konversi model ke format TensorFlow Lite (.tflite)...")

    converter = tf.lite.TFLiteConverter.from_keras_model(model_mobilenet_final)
    tflite_model = converter.convert()

    tflite_model_path = os.path.join(output_dir, 'model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model TFLite telah disimpan di: {tflite_model_path}")

    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")
    print(f"File label telah disimpan di: {labels_path}")

    print(f"\nPROSES LENGKAP SELESAI UNTUK DROPOUT = {dropout_rate}.")

print(f"\n{'='*80}")
print("SEMUA EKSPERIMEN DROPOUT TELAH SELESAI.")
print(f"Semua hasil disimpan di direktori: '{base_output_dir}'")
print(f"{'='*80}")

# MobileNetV3 Large Dengan Decay

In [ ]:
# ==============================================================================
# PROSES 2 & 3: PEMBANGUNAN, PELATIHAN, EVALUASI, DAN PENYIMPANAN MODEL
# DENGAN PERBAIKAN UNTUK CUSTOM GENERATOR
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Daftar nilai dropout yang akan diuji
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Direktori utama untuk menyimpan hasil
base_output_dir = '/content/drive/MyDrive/hasil_eksperimen_dropout_mobilenetv3_large_decay'
os.makedirs(base_output_dir, exist_ok=True)

for dropout_rate in dropout_values:
    print(f"\n{'='*80}")
    print(f"MEMULAI EKSPERIMEN DENGAN DROPOUT = {dropout_rate} (MobileNetV3-Large)")
    print(f"{'='*80}\n")

    output_dir = os.path.join(base_output_dir, f'dropout_{dropout_rate}')
    os.makedirs(output_dir, exist_ok=True)

    base_model = MobileNetV3Large(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, seed=SEED_VALUE)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model_mobilenet_final = Model(inputs=inputs, outputs=outputs)

    initial_learning_rate = 0.001

    # Inisialisasi Learning Rate Decay
    decay_steps = len(train_generator)
    decay_rate = 0.96

    lr_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )

    model_mobilenet_final.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Arsitektur Model:")
    model_mobilenet_final.summary()

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\nMemulai pelatihan model dengan dropout = {dropout_rate}...")
    history = model_mobilenet_final.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping], verbose=1)
    print(f"\nPelatihan model dengan dropout = {dropout_rate} telah selesai.")

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1); plt.plot(history.history['accuracy'], label='Akurasi Pelatihan'); plt.plot(history.history['val_accuracy'], label='Akurasi Validasi'); plt.title(f'Grafik Akurasi (Dropout = {dropout_rate})'); plt.xlabel('Epoch'); plt.ylabel('Akurasi'); plt.legend()
    plt.subplot(1, 2, 2); plt.plot(history.history['loss'], label='Loss Pelatihan'); plt.plot(history.history['val_loss'], label='Loss Validasi'); plt.title(f'Grafik Loss (Dropout = {dropout_rate})'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plot_path = os.path.join(output_dir, 'grafik_pelatihan.png'); plt.savefig(plot_path); plt.close(); print(f"Grafik pelatihan telah disimpan di: {plot_path}")

    print("\nMelakukan evaluasi final pada data tes...")
    test_loss, test_accuracy = model_mobilenet_final.evaluate(test_generator); print(f"Akurasi pada data tes (Dropout = {dropout_rate}): {test_accuracy:.4f}")

    y_pred_probs = model_mobilenet_final.predict(test_generator); y_pred = np.argmax(y_pred_probs, axis=1); y_true = np.array(test_labels)

    print("\nLaporan Klasifikasi pada Data Tes:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True); report_df = pd.DataFrame(report).transpose(); report_path = os.path.join(output_dir, 'classification_report.csv'); report_df.to_csv(report_path); print(classification_report(y_true, y_pred, target_names=class_names)); print(f"Laporan klasifikasi telah disimpan di: {report_path}")
    except Exception as e:
        print(f"Tidak dapat membuat laporan klasifikasi: {e}")

    print("\nConfusion Matrix pada Data Tes:")
    cm = confusion_matrix(y_true, y_pred); plt.figure(figsize=(10, 8)); sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names); plt.title(f'Confusion Matrix (Dropout = {dropout_rate})'); plt.ylabel('Kelas Sebenarnya'); plt.xlabel('Kelas Prediksi'); cm_path = os.path.join(output_dir, 'confusion_matrix.png'); plt.savefig(cm_path); plt.close(); print(f"Confusion matrix telah disimpan di: {cm_path}")

    print("\nMemulai konversi model ke format TensorFlow Lite (.tflite)...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model_mobilenet_final)
    tflite_model = converter.convert()
    tflite_model_path = os.path.join(output_dir, 'model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model TFLite telah disimpan di: {tflite_model_path}")

    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")
    print(f"File label telah disimpan di: {labels_path}")

    print(f"\nPROSES LENGKAP SELESAI UNTUK DROPOUT = {dropout_rate}.")

print(f"\n{'='*80}")
print("SEMUA EKSPERIMEN DROPOUT TELAH SELESAI.")
print(f"Semua hasil disimpan di direktori: '{base_output_dir}'")
print(f"{'='*80}")

# Data Loading (MobileNetV3 Small)

In [ ]:
# ==============================================================================
# PROSES 1: PERSIAPAN DATA (KHUSUS UNTUK MOBILENETV3)
# ==============================================================================

# Langkah 1.1: Install Library
!pip install -q albumentations

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from google.colab import drive
from collections import Counter
import albumentations as A
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import random

# Langkah 1.2: Mount Google Drive
drive.mount('/content/drive')

# Langkah 1.3: Definisi Path dan Parameter
DATASET_DIR = '/content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32


SEED_VALUE = 42
def set_seed(seed=42):
  """
  Mengatur seed dan menonaktifkan parallelism untuk hasil yang fully deterministic.
  """
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

  # Menjalankan operasi TF pada satu thread untuk menghilangkan randomness dari parallelism
  tf.config.threading.set_inter_op_parallelism_threads(1)
  tf.config.threading.set_intra_op_parallelism_threads(1)

  print(f"Random seed set to {seed} and parallelism disabled for reproducibility.")

set_seed(SEED_VALUE)

# Langkah 1.4: Ambil Nama Kelas dari Struktur Direktori Train
train_base_dir = os.path.join(DATASET_DIR, 'train')
class_dirs = sorted([d for d in os.listdir(train_base_dir) if os.path.isdir(os.path.join(train_base_dir, d))])
class_map = {name: i for i, name in enumerate(class_dirs)}

class_names = class_dirs
print(f"Nama kelas: {class_names}")
NUM_CLASSES = len(class_names)
print(f"Jumlah kelas: {NUM_CLASSES}")

# Global lists untuk verifikasi distribusi data
train_labels, val_labels, test_labels = [], [], []

# Langkah 1.5: Transformasi Gambar
basic_transform = A.Compose([
    A.Resize(height=IMAGE_SIZE[0], width=IMAGE_SIZE[1])
])

# Langkah 1.6: Custom Data Generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, class_map, batch_size, transform, num_classes, shuffle=True, store_global_labels=None):
        self.directory = directory
        self.class_map = class_map
        self.batch_size = batch_size
        self.transform = transform
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.image_paths = []
        self.labels = []
        self._load_image_paths()
        self.on_epoch_end()

        if store_global_labels is not None:
            store_global_labels.extend(self.labels)

    def _load_image_paths(self):
        """Memuat path gambar dari direktori yang diberikan."""
        print(f"Loading images from: {self.directory}")
        for class_name in sorted(self.class_map.keys()):
            class_path = os.path.join(self.directory, class_name)
            if os.path.isdir(class_path):
                class_idx = self.class_map[class_name]
                for fname in sorted(os.listdir(class_path)):
                    if fname.lower().endswith('.jpg'):
                        fpath = os.path.join(class_path, fname)
                        self.image_paths.append(fpath)
                        self.labels.append(class_idx)

    def __len__(self):
        """Menghitung jumlah batch per epoch."""
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        """Menghasilkan satu batch data."""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_paths = [self.image_paths[i] for i in indexes]
        batch_labels = [self.labels[i] for i in indexes]
        batch_images = []

        for path in batch_paths:
            img = Image.open(path).convert('RGB')
            img_cv = np.array(img)
            transformed = self.transform(image=img_cv) # Resize Gambar
            batch_images.append(transformed['image'])

        batch_images = preprocess_input(np.array(batch_images, dtype=np.float32))
        batch_labels = tf.keras.utils.to_categorical(batch_labels, num_classes=self.num_classes)
        return batch_images, batch_labels

    def on_epoch_end(self):
        """Mengacak indeks di akhir setiap epoch."""
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Langkah 1.7: Inisialisasi Generator
train_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'train'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=True,
    store_global_labels=train_labels
)

val_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'val'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=False,
    store_global_labels=val_labels
)

test_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'test'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=False,
    store_global_labels=test_labels
)

print(f"\nTotal gambar train: {len(train_generator.image_paths)}")
print(f"Total gambar validation: {len(val_generator.image_paths)}")
print(f"Total gambar test: {len(test_generator.image_paths)}")

print("\nPROSES 1 SELESAI: Data MobileNetV3 siap dari direktori yang sudah di-augmentasi.")


# ==============================================================================
# Verifikasi Distribusi Data di Setiap Split
# ==============================================================================
print("\nVerifikasi Distribusi Data:")

# Menghitung jumlah kemunculan setiap label di setiap set
train_counts = Counter(train_labels)
val_counts = Counter(val_labels)
test_counts = Counter(test_labels)

# Membuat tabel untuk visualisasi yang mudah dibaca
distribusi_data = {
    "Kelas": class_names,
    "Jumlah Train": [train_counts.get(i, 0) for i in range(len(class_names))],
    "Jumlah Validasi": [val_counts.get(i, 0) for i in range(len(class_names))],
    "Jumlah Test": [test_counts.get(i, 0) for i in range(len(class_names))],
}
df_distribusi = pd.DataFrame(distribusi_data)
df_distribusi["Total per Kelas"] = df_distribusi["Jumlah Train"] + df_distribusi["Jumlah Validasi"] + df_distribusi["Jumlah Test"]

# Menambahkan baris total di bagian bawah
total_row = {
    "Kelas": "TOTAL",
    "Jumlah Train": sum(df_distribusi["Jumlah Train"]),
    "Jumlah Validasi": sum(df_distribusi["Jumlah Validasi"]),
    "Jumlah Test": sum(df_distribusi["Jumlah Test"]),
    "Total per Kelas": sum(df_distribusi["Total per Kelas"])
}
df_distribusi = pd.concat([df_distribusi, pd.DataFrame([total_row])], ignore_index=True)

print("Distribusi jumlah data per kelas di setiap split:")
print(df_distribusi.to_string())

# MobileNetV3 Small Tanpa Decay

In [ ]:
# ==============================================================================
# PROSES 2 & 3: PEMBANGUNAN, PELATIHAN, EVALUASI, DAN PENYIMPANAN MODEL
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV3Small
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Daftar nilai dropout yang akan diuji
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Direktori utama untuk menyimpan hasil
base_output_dir = '/content/drive/MyDrive/hasil_eksperimen_dropout_mobilenetv3_small'
os.makedirs(base_output_dir, exist_ok=True)

for dropout_rate in dropout_values:
    print(f"\n{'='*80}")
    print(f"MEMULAI EKSPERIMEN DENGAN DROPOUT = {dropout_rate} (MobileNetV3-Small)")
    print(f"{'='*80}\n")

    # Membuat sub-direktori untuk setiap nilai dropout
    output_dir = os.path.join(base_output_dir, f'dropout_{dropout_rate}')
    os.makedirs(output_dir, exist_ok=True)

    # --- PROSES 2: Pembangunan Arsitektur ---
    base_model = MobileNetV3Small(
        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, seed=SEED_VALUE)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model_mobilenet_final = Model(inputs=inputs, outputs=outputs)
    model_mobilenet_final.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Arsitektur Model:")
    model_mobilenet_final.summary()

    # --- PROSES 3: Pelatihan dan Evaluasi ---
    # Langkah 3.1: Melatih Model
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\nMemulai pelatihan model dengan dropout = {dropout_rate}...")
    history = model_mobilenet_final.fit(
        train_generator,
        epochs=50,
        validation_data=val_generator,
        callbacks=[early_stopping],
        verbose=1
    )
    print(f"\nPelatihan model dengan dropout = {dropout_rate} telah selesai.")

    # Langkah 3.2: Visualisasi dan Simpan Plot
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Akurasi Pelatihan')
    plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
    plt.title(f'Grafik Akurasi (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Akurasi'); plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Loss Pelatihan')
    plt.plot(history.history['val_loss'], label='Loss Validasi')
    plt.title(f'Grafik Loss (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plot_path = os.path.join(output_dir, 'grafik_pelatihan.png')
    plt.savefig(plot_path)
    plt.close()
    print(f"Grafik pelatihan telah disimpan di: {plot_path}")

    # Langkah 3.3: Evaluasi Final dan Simpan Metrik
    print("\nMelakukan evaluasi final pada data tes...")
    test_loss, test_accuracy = model_mobilenet_final.evaluate(test_generator)
    print(f"Akurasi pada data tes (Dropout = {dropout_rate}): {test_accuracy:.4f}")

    y_pred_probs = model_mobilenet_final.predict(test_generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.array(test_labels)

    print("\nLaporan Klasifikasi pada Data Tes:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_path = os.path.join(output_dir, 'classification_report.csv')
        report_df.to_csv(report_path)
        print(classification_report(y_true, y_pred, target_names=class_names))
        print(f"Laporan klasifikasi telah disimpan di: {report_path}")
    except Exception as e:
        print(f"Tidak dapat membuat laporan klasifikasi: {e}")

    print("\nConfusion Matrix pada Data Tes:")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix (Dropout = {dropout_rate})')
    plt.ylabel('Kelas Sebenarnya'); plt.xlabel('Kelas Prediksi')
    cm_path = os.path.join(output_dir, 'confusion_matrix.png')
    plt.savefig(cm_path)
    plt.close()
    print(f"Confusion matrix telah disimpan di: {cm_path}")

    # Langkah 3.4: Konversi dan Simpan Model untuk Flutter
    print("\nMemulai konversi model ke format TensorFlow Lite (.tflite)...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model_mobilenet_final)
    tflite_model = converter.convert()

    tflite_model_path = os.path.join(output_dir, 'model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model TFLite telah disimpan di: {tflite_model_path}")

    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")
    print(f"File label telah disimpan di: {labels_path}")

    print(f"\nPROSES LENGKAP SELESAI UNTUK DROPOUT = {dropout_rate}.")

print(f"\n{'='*80}")
print("SEMUA EKSPERIMEN DROPOUT TELAH SELESAI.")
print(f"Semua hasil disimpan di direktori: '{base_output_dir}'")
print(f"{'='*80}")

# MobileNetV3 Small Dengan Decay

In [ ]:
# ==============================================================================
# PROSES 2 & 3: PEMBANGUNAN, PELATIHAN, EVALUASI, DAN PENYIMPANAN MODEL
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Daftar nilai dropout yang akan diuji
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Direktori utama untuk menyimpan hasil
base_output_dir = '/content/drive/MyDrive/hasil_eksperimen_dropout_mobilenetv3_small_decay'
os.makedirs(base_output_dir, exist_ok=True)

for dropout_rate in dropout_values:
    print(f"\n{'='*80}")
    print(f"MEMULAI EKSPERIMEN DENGAN DROPOUT = {dropout_rate} (MobileNetV3-Small)")
    print(f"{'='*80}\n")

    # Membuat sub-direktori untuk setiap nilai dropout
    output_dir = os.path.join(base_output_dir, f'dropout_{dropout_rate}')
    os.makedirs(output_dir, exist_ok=True)

    # --- PROSES 2: Pembangunan Arsitektur ---
    base_model = MobileNetV3Small(
        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, seed=SEED_VALUE)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model_mobilenet_final = Model(inputs=inputs, outputs=outputs)

    # Inisialisasi Decay
    initial_learning_rate = 0.001
    decay_steps = len(train_generator)
    decay_rate = 0.96

    lr_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )

    # Perubahan 3: Gunakan schedule pada optimizer
    model_mobilenet_final.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Arsitektur Model:")
    model_mobilenet_final.summary()

    # --- PROSES 3: Pelatihan dan Evaluasi ---
    # Langkah 3.1: Melatih Model
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\nMemulai pelatihan model dengan dropout = {dropout_rate}...")
    history = model_mobilenet_final.fit(
        train_generator,
        epochs=50,
        validation_data=val_generator,
        callbacks=[early_stopping],
        verbose=1
    )
    print(f"\nPelatihan model dengan dropout = {dropout_rate} telah selesai.")

    # Langkah 3.2: Visualisasi dan Simpan Plot
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Akurasi Pelatihan')
    plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
    plt.title(f'Grafik Akurasi (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Akurasi'); plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Loss Pelatihan')
    plt.plot(history.history['val_loss'], label='Loss Validasi')
    plt.title(f'Grafik Loss (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plot_path = os.path.join(output_dir, 'grafik_pelatihan.png')
    plt.savefig(plot_path)
    plt.close()
    print(f"Grafik pelatihan telah disimpan di: {plot_path}")

    # Langkah 3.3: Evaluasi Final dan Simpan Metrik
    print("\nMelakukan evaluasi final pada data tes...")
    test_loss, test_accuracy = model_mobilenet_final.evaluate(test_generator)
    print(f"Akurasi pada data tes (Dropout = {dropout_rate}): {test_accuracy:.4f}")

    y_pred_probs = model_mobilenet_final.predict(test_generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.array(test_labels)

    print("\nLaporan Klasifikasi pada Data Tes:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_path = os.path.join(output_dir, 'classification_report.csv')
        report_df.to_csv(report_path)
        print(classification_report(y_true, y_pred, target_names=class_names))
        print(f"Laporan klasifikasi telah disimpan di: {report_path}")
    except Exception as e:
        print(f"Tidak dapat membuat laporan klasifikasi: {e}")

    print("\nConfusion Matrix pada Data Tes:")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix (Dropout = {dropout_rate})')
    plt.ylabel('Kelas Sebenarnya'); plt.xlabel('Kelas Prediksi')
    cm_path = os.path.join(output_dir, 'confusion_matrix.png')
    plt.savefig(cm_path)
    plt.close()
    print(f"Confusion matrix telah disimpan di: {cm_path}")

    # Langkah 3.4: Konversi dan Simpan Model untuk Flutter
    print("\nMemulai konversi model ke format TensorFlow Lite (.tflite)...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model_mobilenet_final)
    tflite_model = converter.convert()

    tflite_model_path = os.path.join(output_dir, 'model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model TFLite telah disimpan di: {tflite_model_path}")

    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")
    print(f"File label telah disimpan di: {labels_path}")

    print(f"\nPROSES LENGKAP SELESAI UNTUK DROPOUT = {dropout_rate}.")

print(f"\n{'='*80}")
print("SEMUA EKSPERIMEN DROPOUT TELAH SELESAI.")
print(f"Semua hasil disimpan di direktori: '{base_output_dir}'")
print(f"{'='*80}")

# Data Loading (MobileNetV2)

In [ ]:
# ==============================================================================
# PROSES 1: PERSIAPAN DATA (KHUSUS UNTUK MOBILENETV2)
# ==============================================================================

# Langkah 1.1: Install Library
!pip install -q albumentations

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from google.colab import drive
from collections import Counter
import albumentations as A
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import random

# Langkah 1.2: Mount Google Drive
drive.mount('/content/drive')

# Langkah 1.3: Definisi Path dan Parameter
DATASET_DIR = '/content/drive/MyDrive/Dataset_Skripsi_Augmented_200_Kran_15'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32


SEED_VALUE = 42
def set_seed(seed=42):
  """
  Mengatur seed dan menonaktifkan parallelism untuk hasil yang fully deterministic.
  """
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

  # Menjalankan operasi TF pada satu thread untuk menghilangkan randomness dari parallelism
  tf.config.threading.set_inter_op_parallelism_threads(1)
  tf.config.threading.set_intra_op_parallelism_threads(1)

  print(f"Random seed set to {seed} and parallelism disabled for reproducibility.")

set_seed(SEED_VALUE)

# Langkah 1.4: Ambil Nama Kelas dari Struktur Direktori Train
train_base_dir = os.path.join(DATASET_DIR, 'train')
class_dirs = sorted([d for d in os.listdir(train_base_dir) if os.path.isdir(os.path.join(train_base_dir, d))])
class_map = {name: i for i, name in enumerate(class_dirs)}

class_names = class_dirs
print(f"Nama kelas: {class_names}")
NUM_CLASSES = len(class_names)
print(f"Jumlah kelas: {NUM_CLASSES}")

# Global lists untuk verifikasi distribusi data
train_labels, val_labels, test_labels = [], [], []

# Langkah 1.5: Transformasi Gambar
basic_transform = A.Compose([
    A.Resize(height=IMAGE_SIZE[0], width=IMAGE_SIZE[1])
])

# Langkah 1.6: Custom Data Generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, class_map, batch_size, transform, num_classes, shuffle=True, store_global_labels=None):
        self.directory = directory
        self.class_map = class_map
        self.batch_size = batch_size
        self.transform = transform
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.image_paths = []
        self.labels = []
        self._load_image_paths()
        self.on_epoch_end()

        if store_global_labels is not None:
            store_global_labels.extend(self.labels)

    def _load_image_paths(self):
        """Memuat path gambar dari direktori yang diberikan."""
        print(f"Loading images from: {self.directory}")
        for class_name in sorted(self.class_map.keys()):
            class_path = os.path.join(self.directory, class_name)
            if os.path.isdir(class_path):
                class_idx = self.class_map[class_name]
                for fname in sorted(os.listdir(class_path)):
                    if fname.lower().endswith('.jpg'):
                        fpath = os.path.join(class_path, fname)
                        self.image_paths.append(fpath)
                        self.labels.append(class_idx)

    def __len__(self):
        """Menghitung jumlah batch per epoch."""
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        """Menghasilkan satu batch data."""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_paths = [self.image_paths[i] for i in indexes]
        batch_labels = [self.labels[i] for i in indexes]
        batch_images = []

        for path in batch_paths:
            img = Image.open(path).convert('RGB')
            img_cv = np.array(img)
            transformed = self.transform(image=img_cv) # Resize Gambar
            batch_images.append(transformed['image'])

        batch_images = preprocess_input(np.array(batch_images, dtype=np.float32))
        batch_labels = tf.keras.utils.to_categorical(batch_labels, num_classes=self.num_classes)
        return batch_images, batch_labels

    def on_epoch_end(self):
        """Mengacak indeks di akhir setiap epoch."""
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Langkah 1.7: Inisialisasi Generator
train_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'train'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=True,
    store_global_labels=train_labels
)

val_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'val'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=False,
    store_global_labels=val_labels
)

test_generator = CustomDataGenerator(
    directory=os.path.join(DATASET_DIR, 'test'),
    class_map=class_map,
    batch_size=BATCH_SIZE,
    transform=basic_transform,
    num_classes=NUM_CLASSES,
    shuffle=False,
    store_global_labels=test_labels
)

print(f"\nTotal gambar train: {len(train_generator.image_paths)}")
print(f"Total gambar validation: {len(val_generator.image_paths)}")
print(f"Total gambar test: {len(test_generator.image_paths)}")

print("\nPROSES 1 SELESAI: Data MobileNetV2 siap dari direktori yang sudah di-augmentasi.")


# ==============================================================================
# Verifikasi Distribusi Data di Setiap Split
# ==============================================================================
print("\nVerifikasi Distribusi Data:")

# Menghitung jumlah kemunculan setiap label di setiap set
train_counts = Counter(train_labels)
val_counts = Counter(val_labels)
test_counts = Counter(test_labels)

# Membuat tabel untuk visualisasi yang mudah dibaca
distribusi_data = {
    "Kelas": class_names,
    "Jumlah Train": [train_counts.get(i, 0) for i in range(len(class_names))],
    "Jumlah Validasi": [val_counts.get(i, 0) for i in range(len(class_names))],
    "Jumlah Test": [test_counts.get(i, 0) for i in range(len(class_names))],
}
df_distribusi = pd.DataFrame(distribusi_data)
df_distribusi["Total per Kelas"] = df_distribusi["Jumlah Train"] + df_distribusi["Jumlah Validasi"] + df_distribusi["Jumlah Test"]

# Menambahkan baris total di bagian bawah
total_row = {
    "Kelas": "TOTAL",
    "Jumlah Train": sum(df_distribusi["Jumlah Train"]),
    "Jumlah Validasi": sum(df_distribusi["Jumlah Validasi"]),
    "Jumlah Test": sum(df_distribusi["Jumlah Test"]),
    "Total per Kelas": sum(df_distribusi["Total per Kelas"])
}
df_distribusi = pd.concat([df_distribusi, pd.DataFrame([total_row])], ignore_index=True)

print("Distribusi jumlah data per kelas di setiap split:")
print(df_distribusi.to_string())

# MobileNetV2 Tanpa Decay

In [ ]:
# ==============================================================================
# PROSES 2 & 3: PEMBANGUNAN, PELATIHAN, EVALUASI, DAN PENYIMPANAN MODEL
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Daftar nilai dropout yang akan diuji
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Direktori utama untuk menyimpan hasil
base_output_dir = '/content/drive/MyDrive/hasil_eksperimen_dropout_mobilenetv2'
os.makedirs(base_output_dir, exist_ok=True)

for dropout_rate in dropout_values:
    print(f"\n{'='*80}")
    print(f"MEMULAI EKSPERIMEN DENGAN DROPOUT = {dropout_rate} (MobileNetV2)")
    print(f"{'='*80}\n")

    # Membuat sub-direktori untuk setiap nilai dropout
    output_dir = os.path.join(base_output_dir, f'dropout_{dropout_rate}')
    os.makedirs(output_dir, exist_ok=True)

    # --- PROSES 2: Pembangunan Arsitektur ---
    base_model = MobileNetV2(
        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, seed=SEED_VALUE)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model_mobilenet_final = Model(inputs=inputs, outputs=outputs)
    model_mobilenet_final.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Arsitektur Model:")
    model_mobilenet_final.summary()

    # --- PROSES 3: Pelatihan dan Evaluasi ---
    # Langkah 3.1: Melatih Model
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\nMemulai pelatihan model dengan dropout = {dropout_rate}...")
    history = model_mobilenet_final.fit(
        train_generator,
        epochs=50,
        validation_data=val_generator,
        callbacks=[early_stopping],
        verbose=1
    )
    print(f"\nPelatihan model dengan dropout = {dropout_rate} telah selesai.")

    # Langkah 3.2: Visualisasi dan Simpan Plot
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Akurasi Pelatihan')
    plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
    plt.title(f'Grafik Akurasi (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Akurasi'); plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Loss Pelatihan')
    plt.plot(history.history['val_loss'], label='Loss Validasi')
    plt.title(f'Grafik Loss (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plot_path = os.path.join(output_dir, 'grafik_pelatihan.png')
    plt.savefig(plot_path)
    plt.close()
    print(f"Grafik pelatihan telah disimpan di: {plot_path}")

    # Langkah 3.3: Evaluasi Final dan Simpan Metrik
    print("\nMelakukan evaluasi final pada data tes...")
    test_loss, test_accuracy = model_mobilenet_final.evaluate(test_generator)
    print(f"Akurasi pada data tes (Dropout = {dropout_rate}): {test_accuracy:.4f}")

    y_pred_probs = model_mobilenet_final.predict(test_generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.array(test_labels)

    print("\nLaporan Klasifikasi pada Data Tes:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_path = os.path.join(output_dir, 'classification_report.csv')
        report_df.to_csv(report_path)
        print(classification_report(y_true, y_pred, target_names=class_names))
        print(f"Laporan klasifikasi telah disimpan di: {report_path}")
    except Exception as e:
        print(f"Tidak dapat membuat laporan klasifikasi: {e}")

    print("\nConfusion Matrix pada Data Tes:")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix (Dropout = {dropout_rate})')
    plt.ylabel('Kelas Sebenarnya'); plt.xlabel('Kelas Prediksi')
    cm_path = os.path.join(output_dir, 'confusion_matrix.png')
    plt.savefig(cm_path)
    plt.close()
    print(f"Confusion matrix telah disimpan di: {cm_path}")

    # Langkah 3.4: Konversi dan Simpan Model untuk Flutter
    print("\nMemulai konversi model ke format TensorFlow Lite (.tflite)...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model_mobilenet_final)
    tflite_model = converter.convert()

    tflite_model_path = os.path.join(output_dir, 'model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model TFLite telah disimpan di: {tflite_model_path}")

    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")
    print(f"File label telah disimpan di: {labels_path}")

    print(f"\nPROSES LENGKAP SELESAI UNTUK DROPOUT = {dropout_rate}.")

print(f"\n{'='*80}")
print("SEMUA EKSPERIMEN DROPOUT TELAH SELESAI.")
print(f"Semua hasil disimpan di direktori: '{base_output_dir}'")
print(f"{'='*80}")

# MobileNetV2 Dengan Decay

In [ ]:
# ==============================================================================
# PROSES 2 & 3: PEMBANGUNAN, PELATIHAN, EVALUASI, DAN PENYIMPANAN MODEL
# ==============================================================================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Daftar nilai dropout yang akan diuji
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Direktori utama untuk menyimpan hasil
base_output_dir = '/content/drive/MyDrive/hasil_eksperimen_dropout_mobilenetv2_decay'
os.makedirs(base_output_dir, exist_ok=True)

for dropout_rate in dropout_values:
    print(f"\n{'='*80}")
    print(f"MEMULAI EKSPERIMEN DENGAN DROPOUT = {dropout_rate} (MobileNetV2)")
    print(f"{'='*80}\n")

    # Membuat sub-direktori untuk setiap nilai dropout
    output_dir = os.path.join(base_output_dir, f'dropout_{dropout_rate}')
    os.makedirs(output_dir, exist_ok=True)

    # --- PROSES 2: Pembangunan Arsitektur ---
    base_model = MobileNetV2(
        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, seed=SEED_VALUE)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model_mobilenet_final = Model(inputs=inputs, outputs=outputs)

    # Inisialisasi Decay
    initial_learning_rate = 0.001
    decay_steps = len(train_generator)
    decay_rate = 0.96

    lr_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )

    model_mobilenet_final.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Arsitektur Model:")
    model_mobilenet_final.summary()

    # --- PROSES 3: Pelatihan dan Evaluasi ---
    # Langkah 3.1: Melatih Model
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\nMemulai pelatihan model dengan dropout = {dropout_rate}...")
    history = model_mobilenet_final.fit(
        train_generator,
        epochs=50,
        validation_data=val_generator,
        callbacks=[early_stopping],
        verbose=1
    )
    print(f"\nPelatihan model dengan dropout = {dropout_rate} telah selesai.")

    # Langkah 3.2: Visualisasi dan Simpan Plot
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Akurasi Pelatihan')
    plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
    plt.title(f'Grafik Akurasi (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Akurasi'); plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Loss Pelatihan')
    plt.plot(history.history['val_loss'], label='Loss Validasi')
    plt.title(f'Grafik Loss (Dropout = {dropout_rate})')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plot_path = os.path.join(output_dir, 'grafik_pelatihan.png')
    plt.savefig(plot_path)
    plt.close()
    print(f"Grafik pelatihan telah disimpan di: {plot_path}")

    # Langkah 3.3: Evaluasi Final dan Simpan Metrik
    print("\nMelakukan evaluasi final pada data tes...")
    test_loss, test_accuracy = model_mobilenet_final.evaluate(test_generator)
    print(f"Akurasi pada data tes (Dropout = {dropout_rate}): {test_accuracy:.4f}")

    y_pred_probs = model_mobilenet_final.predict(test_generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.array(test_labels)

    print("\nLaporan Klasifikasi pada Data Tes:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_path = os.path.join(output_dir, 'classification_report.csv')
        report_df.to_csv(report_path)
        print(classification_report(y_true, y_pred, target_names=class_names))
        print(f"Laporan klasifikasi telah disimpan di: {report_path}")
    except Exception as e:
        print(f"Tidak dapat membuat laporan klasifikasi: {e}")

    print("\nConfusion Matrix pada Data Tes:")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix (Dropout = {dropout_rate})')
    plt.ylabel('Kelas Sebenarnya'); plt.xlabel('Kelas Prediksi')
    cm_path = os.path.join(output_dir, 'confusion_matrix.png')
    plt.savefig(cm_path)
    plt.close()
    print(f"Confusion matrix telah disimpan di: {cm_path}")

    # Langkah 3.4: Konversi dan Simpan Model untuk Flutter
    print("\nMemulai konversi model ke format TensorFlow Lite (.tflite)...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model_mobilenet_final)
    tflite_model = converter.convert()

    tflite_model_path = os.path.join(output_dir, 'model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model TFLite telah disimpan di: {tflite_model_path}")

    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")
    print(f"File label telah disimpan di: {labels_path}")

    print(f"\nPROSES LENGKAP SELESAI UNTUK DROPOUT = {dropout_rate}.")

print(f"\n{'='*80}")
print("SEMUA EKSPERIMEN DROPOUT TELAH SELESAI.")
print(f"Semua hasil disimpan di direktori: '{base_output_dir}'")
print(f"{'='*80}")